In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
from csv import writer
import math
import cmath
import pickle

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [10]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16])
    out= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19])
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)

[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


In [15]:
from sklearn.decomposition import PCA
import warnings
scaler_obj1=PCA(svd_solver='full')
scaler_obj2=PCA(svd_solver='full')
X1=scaler_obj1.fit_transform(A1)
Y1=scaler_obj2.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

X1=X1[:,np.newaxis,:]
Y1=Y1[:,np.newaxis,:]

In [16]:
from keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [17]:
model = Sequential()
model.add(keras.Input(shape=(1,5)))
model.add(tf.keras.layers.GRU(14,activation="tanh",use_bias=True,kernel_initializer="glorot_uniform",bias_initializer="zeros", 
                                kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
                                bias_regularizer=keras.regularizers.l2(1e-4),
                                activity_regularizer=keras.regularizers.l2(1e-5)))
model.add(keras.layers.Dropout(.1))
model.add(Dense(7))
model.add(keras.layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True,
                                beta_initializer="zeros",gamma_initializer="ones",
                                moving_mean_initializer="zeros",moving_variance_initializer="ones",trainable=True))
model.add(keras.layers.ReLU())
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),loss='mse',metrics=['accuracy','mse','mae',rmse])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 14)                882       
                                                                 
 dropout_1 (Dropout)         (None, 14)                0         
                                                                 
 dense_1 (Dense)             (None, 7)                 105       
                                                                 
 batch_normalization_1 (Batc  (None, 7)                28        
 hNormalization)                                                 
                                                                 
 re_lu_1 (ReLU)              (None, 7)                 0         
                                                                 
Total params: 1,015
Trainable params: 1,001
Non-trainable params: 14
___________________________________________________

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=42)

history2 = model.fit(x_train,y_train,batch_size=256,epochs=50, validation_split=0.1)

Epoch 1/50
4563/4563 [==============================] - 55s 12ms/step - loss: 2296374272.0000 - accuracy: 0.1494 - mse: 2296471552.0000 - mae: 96.6970 - rmse: 197.0939 - val_loss: 30779.7090 - val_accuracy: 0.1411 - val_mse: 30779.7461 - val_mae: 72.9065 - val_rmse: 137.4894
Epoch 2/50
4563/4563 [==============================] - 53s 12ms/step - loss: 2296356864.0000 - accuracy: 0.1428 - mse: 2296454144.0000 - mae: 96.6854 - rmse: 197.0893 - val_loss: 30778.8359 - val_accuracy: 0.1396 - val_mse: 30778.8770 - val_mae: 72.8932 - val_rmse: 137.4845
Epoch 3/50
4563/4563 [==============================] - 54s 12ms/step - loss: 2296363264.0000 - accuracy: 0.1472 - mse: 2296460544.0000 - mae: 96.6758 - rmse: 197.0874 - val_loss: 30778.1562 - val_accuracy: 0.1602 - val_mse: 30778.1973 - val_mae: 72.8822 - val_rmse: 137.4800
Epoch 4/50
4563/4563 [==============================] - 53s 12ms/step - loss: 2296357632.0000 - accuracy: 0.1589 - mse: 2296455168.0000 - mae: 96.6666 - rmse: 197.0860 - va

Epoch 31/50
4563/4563 [==============================] - 53s 12ms/step - loss: 2296374528.0000 - accuracy: 0.1705 - mse: 2296471808.0000 - mae: 96.6057 - rmse: 197.0713 - val_loss: 30776.9941 - val_accuracy: 0.1473 - val_mse: 30777.0371 - val_mae: 72.8306 - val_rmse: 137.4686
Epoch 32/50
4563/4563 [==============================] - 53s 12ms/step - loss: 2296372480.0000 - accuracy: 0.1707 - mse: 2296469760.0000 - mae: 96.6051 - rmse: 197.0704 - val_loss: 30777.0137 - val_accuracy: 0.1469 - val_mse: 30777.0527 - val_mae: 72.8308 - val_rmse: 137.4686
Epoch 33/50
4563/4563 [==============================] - 53s 12ms/step - loss: 2296357120.0000 - accuracy: 0.1717 - mse: 2296454400.0000 - mae: 96.6035 - rmse: 197.0668 - val_loss: 30776.9824 - val_accuracy: 0.1472 - val_mse: 30777.0254 - val_mae: 72.8304 - val_rmse: 137.4683
Epoch 34/50
4563/4563 [==============================] - 54s 12ms/step - loss: 2296357376.0000 - accuracy: 0.1719 - mse: 2296454656.0000 - mae: 96.6044 - rmse: 197.0705 

In [20]:
model.evaluate(x_test,y_test)

13518/13518 [==============================] - 180s 13ms/step - loss: 1432071.7500 - accuracy: 0.1475 - mse: 1432085.3750 - mae: 74.2265 - rmse: 141.0122


[1432071.75,
 0.1475204974412918,
 1432085.375,
 74.22654724121094,
 141.01223754882812]

In [21]:
model.evaluate(x_train,y_train)

40554/40554 [==============================] - 537s 13ms/step - loss: 2066725120.0000 - accuracy: 0.1478 - mse: 2066737920.0000 - mae: 94.2109 - rmse: 191.0949


[2066725120.0,
 0.14784526824951172,
 2066737920.0,
 94.21088409423828,
 191.09494018554688]

In [22]:
df1=pd.DataFrame(history2.history['loss'],columns=["Loss"])
df1=df1.join(pd.DataFrame(history2.history["val_loss"],columns=["Val Loss"]))
df1=df1.join(pd.DataFrame(history2.history["accuracy"],columns=['Accuracy']))
df1=df1.join(pd.DataFrame(history2.history["val_accuracy"],columns=['Val Accuracy']))
df1=df1.join(pd.DataFrame(history2.history["mse"],columns=['MSE']))
df1=df1.join(pd.DataFrame(history2.history["val_mse"],columns=['Val MSE']))
df1=df1.join(pd.DataFrame(history2.history["mae"],columns=['MAE']))
df1=df1.join(pd.DataFrame(history2.history["val_mae"],columns=['Val MAE']))
df1=df1.join(pd.DataFrame(history2.history["rmse"],columns=['RMSE']))
df1=df1.join(pd.DataFrame(history2.history["val_mse"],columns=['Val RMSE']))
df1

,Loss,Val Loss,Accuracy,Val Accuracy,MSE,Val MSE,MAE,Val MAE,RMSE,Val RMSE
0,2.296374e+09,30779.708984,0.149436,0.141140,2.296472e+09,30779.746094,96.697021,72.906517,197.093933,30779.746094
1,2.296357e+09,30778.835938,0.142781,0.139645,2.296454e+09,30778.876953,96.685364,72.893196,197.089279,30778.876953
2,2.296363e+09,30778.156250,0.147191,0.160157,2.296461e+09,30778.197266,96.675766,72.882217,197.087402,30778.197266
3,2.296358e+09,30778.187500,0.158868,0.158688,2.296455e+09,30778.228516,96.666580,72.872559,197.086044,30778.228516
4,2.296365e+09,30778.277344,0.164639,0.158617,2.296462e+09,30778.318359,96.656998,72.862648,197.084885,30778.318359
5,2.296347e+09,30778.253906,0.169908,0.158565,2.296444e+09,30778.294922,96.649445,72.857109,197.085129,30778.294922
6,2.296356e+09,30778.177734,0.174336,0.160683,2.296453e+09,30778.218750,96.643623,72.854111,197.084274,30778.218750
7,2.296358e+09,30777.537109,0.176364,0.162410,2.296455e+09,30777.580078,96.637306,72.847641,197.082428,30777.580078
8,2.296348e+09,30777.650391,0.177316,0.162228,2.296446e+09,30777.693359,96.639694,72.859528,197.080902,30777.693359
9,2.296367e+09,30777.476562,0.177832,0.162489,2.296465e+09,30777.517578,96.636879,72.856461,197.081451,30777.517578


In [23]:
df1.to_excel("GRU_tanh_mse.xlsx")

In [25]:
model_json = model.to_json()
with open("gru_tanh_mse.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("gru_tanh_mse.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=42)

from keras.models import model_from_json
json_file = open('gru_tanh_mse.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("gru_tanh_mse.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),loss='mse',metrics=['accuracy','mse','mae',rmse])
print(loaded_model.evaluate(x_train, y_train, verbose=0))

Loaded model from disk
[2066725120.0, 0.14784526824951172, 2066737920.0, 94.21088409423828, 191.09494018554688]


In [27]:
print(loaded_model.evaluate(x_test,y_test))

13518/13518 [==============================] - 100s 7ms/step - loss: 1432071.7500 - accuracy: 0.1475 - mse: 1432085.3750 - mae: 74.2265 - rmse: 141.0122
[1432071.75, 0.1475204974412918, 1432085.375, 74.22654724121094, 141.01223754882812]


In [28]:
print(loaded_model.evaluate(x_train,y_train))

40554/40554 [==============================] - 299s 7ms/step - loss: 2066725120.0000 - accuracy: 0.1478 - mse: 2066737920.0000 - mae: 94.2109 - rmse: 191.0949
[2066725120.0, 0.14784526824951172, 2066737920.0, 94.21088409423828, 191.09494018554688]


In [ ]:
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=42)

y_test_pred=model.predict(x_test)
y_test_pred

In [ ]:
y_test

In [ ]:
y_test=y_test[:,0]

In [ ]:
from numpy import savetxt
savetxt('gru_tanh_mse_y_test_pred.csv', y_test_pred[:1001], delimiter=',')

In [ ]:
from numpy import savetxt
savetxt('gru_tanh_mse_y_test.csv', y_test[:1001], delimiter=',')

In [ ]:
#completed